# Module 3.2: SIFT 簡化版 (Scale-Invariant Feature Transform)

## 學習目標

本 notebook 將帶你理解 SIFT 的核心概念：

1. **尺度空間 (Scale Space)** 的概念
2. **Gaussian Pyramid** 建構
3. **DoG (Difference of Gaussian)** 計算
4. **極值偵測** 找關鍵點
5. **簡化版描述子** 的概念

**注意**：完整的 SIFT 非常複雜，這裡只實作簡化版本。

## 參考資料
- Lowe, 2004. "Distinctive Image Features from Scale-Invariant Keypoints"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
sys.path.append('../../')

print("環境設定完成！")

In [ ]:
# 基礎函數
def conv2d(image, kernel, padding='same'):
    """2D 卷積"""
    kH, kW = kernel.shape
    if padding == 'same':
        pad_h, pad_w = (kH - 1) // 2, (kW - 1) // 2
    else:
        pad_h, pad_w = 0, 0
    
    if pad_h > 0 or pad_w > 0:
        image = np.pad(image, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant')
    
    H, W = image.shape
    out_H, out_W = H - kH + 1, W - kW + 1
    output = np.zeros((out_H, out_W), dtype=np.float64)
    
    for i in range(out_H):
        for j in range(out_W):
            output[i, j] = np.sum(image[i:i+kH, j:j+kW] * kernel)
    return output

def gaussian_kernel(size, sigma):
    """建立 Gaussian kernel"""
    half = size // 2
    x = np.arange(-half, half + 1)
    X, Y = np.meshgrid(x, x)
    gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
    return gaussian / gaussian.sum()

def gaussian_blur(image, sigma):
    """Gaussian 平滑"""
    size = int(np.ceil(6 * sigma)) | 1
    kernel = gaussian_kernel(size, sigma)
    return conv2d(image, kernel, padding='same')

SOBEL_X = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float64)
SOBEL_Y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=np.float64)

print("基礎函數已載入！")

In [ ]:
# 測試影像
def create_test_image(size=200):
    np.random.seed(42)
    image = np.ones((size, size), dtype=np.float64) * 80
    
    # 不同大小的方塊（測試尺度不變性）
    image[20:50, 20:50] = 200
    image[30:45, 100:115] = 180
    image[80:140, 50:110] = 160
    
    # 圓形
    y, x = np.ogrid[:size, :size]
    mask = (x - 150)**2 + (y - 100)**2 <= 30**2
    image[mask] = 190
    
    noise = np.random.randn(size, size) * 3
    return np.clip(image + noise, 0, 255)

test_image = create_test_image(200)
plt.figure(figsize=(8, 8))
plt.imshow(test_image, cmap='gray', vmin=0, vmax=255)
plt.title('Test Image')
plt.colorbar()
plt.show()

---

# Part 1: 為什麼需要尺度空間？

## 問題

同一個物體在不同距離下會有不同大小：
- 近距離：物體大，細節多
- 遠距離：物體小，細節少

我們希望無論物體大小如何，都能找到相同的關鍵點。

## 解決方案：Scale Space

在多個尺度（不同程度的模糊）下分析影像：

$$
L(x, y, \sigma) = G(x, y, \sigma) * I(x, y)
$$

其中 $G(x, y, \sigma)$ 是 Gaussian kernel，$\sigma$ 控制尺度。

In [ ]:
# 視覺化不同尺度
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

sigmas = [0.5, 1.0, 2.0, 4.0, 8.0]
for ax, sigma in zip(axes, sigmas):
    blurred = gaussian_blur(test_image, sigma)
    ax.imshow(blurred, cmap='gray', vmin=0, vmax=255)
    ax.set_title(f'σ = {sigma}')
    ax.axis('off')

plt.suptitle('Scale Space: Different σ values')
plt.tight_layout()
plt.show()

print("觀察：")
print("- σ 小：保留細節")
print("- σ 大：只保留大結構")
print("- 不同尺度會突顯不同大小的特徵")

---

# Part 2: Gaussian Pyramid

## 結構

Gaussian Pyramid 由多個 **Octave** 組成：
- 每個 Octave 有多個 **Scale**（不同 σ 的 Gaussian blur）
- 每個 Octave 的影像大小減半

## σ 的計算

在 Octave $o$，Scale $s$：

$$
\sigma(o, s) = \sigma_0 \cdot 2^{o + s/S}
$$

其中 $\sigma_0$ 是基準 sigma，$S$ 是每個 octave 的 scale 數量。

In [ ]:
def build_gaussian_pyramid(image, num_octaves=4, num_scales=5, sigma_0=1.6):
    """
    建構 Gaussian Pyramid。
    
    Parameters
    ----------
    image : np.ndarray
        輸入影像
    num_octaves : int
        Octave 數量（影像大小層數）
    num_scales : int
        每個 Octave 的 scale 數量
    sigma_0 : float
        基準 sigma
    
    Returns
    -------
    pyramid : list of list of np.ndarray
        pyramid[octave][scale] 是一個模糊影像
    sigmas : list of list of float
        對應的 sigma 值
    """
    pyramid = []
    sigmas = []
    
    # 計算 sigma 的乘數因子
    k = 2 ** (1.0 / num_scales)
    
    current_image = image.astype(np.float64)
    
    for octave in range(num_octaves):
        octave_images = []
        octave_sigmas = []
        
        for scale in range(num_scales):
            # 計算這個 scale 的 sigma
            sigma = sigma_0 * (k ** scale)
            
            # 對當前 octave 的基礎影像做 blur
            blurred = gaussian_blur(current_image, sigma)
            
            octave_images.append(blurred)
            octave_sigmas.append(sigma)
        
        pyramid.append(octave_images)
        sigmas.append(octave_sigmas)
        
        # 下採樣到下一個 octave
        # 取 octave 中某個 scale 的影像，然後縮小一半
        current_image = current_image[::2, ::2]  # 簡單的下採樣
    
    return pyramid, sigmas

# 建構 pyramid
pyramid, sigmas = build_gaussian_pyramid(test_image, num_octaves=4, num_scales=5)

print("Gaussian Pyramid 結構：")
for o, octave in enumerate(pyramid):
    print(f"  Octave {o}: {len(octave)} scales, size = {octave[0].shape}")
    print(f"    Sigmas: {[f'{s:.2f}' for s in sigmas[o]]}")

In [ ]:
# 視覺化 Pyramid
fig, axes = plt.subplots(len(pyramid), len(pyramid[0]), figsize=(20, 12))

for o, octave in enumerate(pyramid):
    for s, img in enumerate(octave):
        ax = axes[o, s]
        ax.imshow(img, cmap='gray', vmin=0, vmax=255)
        ax.set_title(f'O{o} S{s}\nσ={sigmas[o][s]:.2f}', fontsize=8)
        ax.axis('off')

plt.suptitle('Gaussian Pyramid')
plt.tight_layout()
plt.show()

---

# Part 3: Difference of Gaussian (DoG)

## 概念

**DoG** 是相鄰尺度 Gaussian 影像的差：

$$
D(x, y, \sigma) = L(x, y, k\sigma) - L(x, y, \sigma)
$$

## 為什麼用 DoG？

1. DoG 近似於 **Laplacian of Gaussian (LoG)**
2. LoG 可以偵測 blob（斑點）結構
3. DoG 計算更快

$$
G(x, y, k\sigma) - G(x, y, \sigma) \approx (k-1)\sigma^2 \nabla^2 G
$$

In [ ]:
def compute_dog(pyramid):
    """
    計算 Difference of Gaussian。
    
    Parameters
    ----------
    pyramid : list of list of np.ndarray
        Gaussian pyramid
    
    Returns
    -------
    dog : list of list of np.ndarray
        DoG pyramid
        每個 octave 有 num_scales - 1 個 DoG 影像
    """
    dog = []
    
    for octave in pyramid:
        octave_dog = []
        
        # 相鄰 scale 相減
        for i in range(len(octave) - 1):
            diff = octave[i + 1] - octave[i]
            octave_dog.append(diff)
        
        dog.append(octave_dog)
    
    return dog

# 計算 DoG
dog = compute_dog(pyramid)

print("DoG Pyramid 結構：")
for o, octave in enumerate(dog):
    print(f"  Octave {o}: {len(octave)} DoG images, size = {octave[0].shape}")

In [ ]:
# 視覺化 DoG
fig, axes = plt.subplots(len(dog), len(dog[0]), figsize=(18, 12))

for o, octave in enumerate(dog):
    for s, img in enumerate(octave):
        ax = axes[o, s]
        # DoG 有正有負，用 RdBu colormap
        vmax = np.abs(img).max()
        ax.imshow(img, cmap='RdBu', vmin=-vmax, vmax=vmax)
        ax.set_title(f'O{o} DoG{s}', fontsize=8)
        ax.axis('off')

plt.suptitle('Difference of Gaussian')
plt.tight_layout()
plt.show()

print("觀察：")
print("- DoG 強調邊緣和 blob 結構")
print("- 不同尺度的 DoG 對不同大小的特徵敏感")

---

# Part 4: 極值偵測 (Keypoint Detection)

## 方法

在 DoG 尺度空間中找 **局部極值**：
- 每個點和它的 26 個鄰居比較（當前尺度 8 個 + 上下尺度各 9 個）
- 如果是最大值或最小值，就是潛在的關鍵點

In [ ]:
def is_extremum(dog, octave, scale, y, x):
    """
    檢查 (octave, scale, y, x) 是否是局部極值。
    
    比較當前點和 26 個鄰居：
    - 當前 scale 的 8 個鄰居
    - 上一個 scale 的 9 個鄰居（包含正上方）
    - 下一個 scale 的 9 個鄰居
    """
    center = dog[octave][scale][y, x]
    
    # 收集 26 個鄰居
    neighbors = []
    
    for s in range(scale - 1, scale + 2):  # 三個 scale
        if s < 0 or s >= len(dog[octave]):
            continue
        for dy in range(-1, 2):
            for dx in range(-1, 2):
                if s == scale and dy == 0 and dx == 0:
                    continue  # 跳過自己
                ny, nx = y + dy, x + dx
                H, W = dog[octave][s].shape
                if 0 <= ny < H and 0 <= nx < W:
                    neighbors.append(dog[octave][s][ny, nx])
    
    if len(neighbors) == 0:
        return False
    
    # 是否是極大值或極小值
    is_max = center > max(neighbors)
    is_min = center < min(neighbors)
    
    return is_max or is_min


def find_extrema(dog, contrast_threshold=0.03):
    """
    在 DoG 尺度空間中找極值點。
    
    Parameters
    ----------
    dog : list of list of np.ndarray
        DoG pyramid
    contrast_threshold : float
        對比度閾值（過濾低對比度的點）
    
    Returns
    -------
    keypoints : list of (octave, scale, y, x)
        關鍵點列表
    """
    keypoints = []
    
    for octave in range(len(dog)):
        # 只在中間的 scales 找（需要上下相鄰）
        for scale in range(1, len(dog[octave]) - 1):
            H, W = dog[octave][scale].shape
            
            for y in range(1, H - 1):
                for x in range(1, W - 1):
                    # 檢查對比度
                    if abs(dog[octave][scale][y, x]) < contrast_threshold:
                        continue
                    
                    # 檢查是否是極值
                    if is_extremum(dog, octave, scale, y, x):
                        keypoints.append((octave, scale, y, x))
    
    return keypoints

# 找極值點
keypoints = find_extrema(dog, contrast_threshold=0.02)
print(f"找到 {len(keypoints)} 個潛在關鍵點")

In [ ]:
# 視覺化關鍵點
def visualize_keypoints(image, keypoints, pyramid):
    """
    在原圖上視覺化關鍵點。
    不同 octave 用不同顏色，圓圈大小表示 scale。
    """
    colors = ['red', 'green', 'blue', 'yellow', 'purple']
    
    plt.figure(figsize=(10, 10))
    plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    
    for octave, scale, y, x in keypoints:
        # 換算回原圖座標
        scale_factor = 2 ** octave
        orig_y = y * scale_factor
        orig_x = x * scale_factor
        
        # 圓圈大小根據 octave 和 scale
        radius = (scale + 1) * scale_factor * 2
        
        color = colors[octave % len(colors)]
        circle = plt.Circle((orig_x, orig_y), radius, 
                            fill=False, color=color, linewidth=1)
        plt.gca().add_patch(circle)
    
    plt.title(f'{len(keypoints)} Keypoints Detected')
    plt.axis('off')
    plt.show()

visualize_keypoints(test_image, keypoints, pyramid)

---

# Part 5: 方向賦值 (Orientation Assignment)

## 概念

為每個關鍵點計算一個主方向，讓描述子具有旋轉不變性。

## 方法

1. 在關鍵點周圍取一個區域
2. 計算每個像素的梯度方向
3. 建立方向直方圖（36 bins，每 10°）
4. 直方圖的峰值就是主方向

In [ ]:
def compute_keypoint_orientation(image, y, x, window_size=16, num_bins=36):
    """
    計算關鍵點的主方向。
    
    Parameters
    ----------
    image : np.ndarray
        影像
    y, x : int
        關鍵點座標
    window_size : int
        視窗大小
    num_bins : int
        方向直方圖的 bin 數量
    
    Returns
    -------
    orientation : float
        主方向（度，0-360）
    histogram : np.ndarray
        方向直方圖
    """
    H, W = image.shape
    half = window_size // 2
    
    # 確保在邊界內
    y0 = max(0, y - half)
    y1 = min(H, y + half)
    x0 = max(0, x - half)
    x1 = min(W, x + half)
    
    region = image[y0:y1, x0:x1]
    
    if region.size == 0:
        return 0, np.zeros(num_bins)
    
    # 計算梯度
    Gx = conv2d(region, SOBEL_X, padding='same')
    Gy = conv2d(region, SOBEL_Y, padding='same')
    
    magnitude = np.sqrt(Gx**2 + Gy**2)
    direction = np.arctan2(Gy, Gx)  # -π to π
    direction = np.rad2deg(direction) % 360  # 0 to 360
    
    # 建立 Gaussian 權重
    sigma = window_size / 2
    gy, gx = np.mgrid[0:region.shape[0], 0:region.shape[1]]
    cy, cx = region.shape[0] / 2, region.shape[1] / 2
    weight = np.exp(-((gy - cy)**2 + (gx - cx)**2) / (2 * sigma**2))
    
    # 加權直方圖
    histogram = np.zeros(num_bins)
    bin_width = 360 / num_bins
    
    for i in range(region.shape[0]):
        for j in range(region.shape[1]):
            bin_idx = int(direction[i, j] / bin_width) % num_bins
            histogram[bin_idx] += magnitude[i, j] * weight[i, j]
    
    # 找主方向（最大 bin）
    main_bin = np.argmax(histogram)
    orientation = (main_bin + 0.5) * bin_width
    
    return orientation, histogram

# 測試
if len(keypoints) > 0:
    octave, scale, y, x = keypoints[0]
    img = pyramid[octave][scale]
    ori, hist = compute_keypoint_orientation(img, y, x)
    
    print(f"關鍵點 (O{octave}, S{scale}, {y}, {x}) 的主方向：{ori:.1f}°")
    
    # 視覺化直方圖
    plt.figure(figsize=(10, 4))
    bins = np.arange(36) * 10 + 5  # 每個 bin 的中心
    plt.bar(bins, hist, width=8)
    plt.axvline(ori, color='red', linestyle='--', label=f'Main: {ori:.1f}°')
    plt.xlabel('Direction (degrees)')
    plt.ylabel('Weighted Count')
    plt.title('Orientation Histogram')
    plt.legend()
    plt.show()

---

# Part 6: 簡化版描述子 (Simplified Descriptor)

## SIFT 描述子概念

SIFT 描述子的建構：
1. 在關鍵點周圍取 16×16 區域
2. 分成 4×4 = 16 個子區域（每個 4×4 像素）
3. 每個子區域計算 8-bin 方向直方圖
4. 總共 16×8 = 128 維向量

這裡我們實作簡化版。

In [ ]:
def compute_sift_descriptor(image, y, x, orientation, patch_size=16, grid_size=4, num_bins=8):
    """
    計算簡化版 SIFT 描述子。
    
    Parameters
    ----------
    image : np.ndarray
        影像
    y, x : int
        關鍵點座標
    orientation : float
        主方向（度）
    patch_size : int
        區域大小（16）
    grid_size : int
        網格大小（4×4）
    num_bins : int
        每個子區域的方向 bin 數（8）
    
    Returns
    -------
    descriptor : np.ndarray
        描述子向量（128 維）
    """
    H, W = image.shape
    half = patch_size // 2
    cell_size = patch_size // grid_size
    
    # 確保在邊界內
    if y - half < 0 or y + half >= H or x - half < 0 or x + half >= W:
        return np.zeros(grid_size * grid_size * num_bins)
    
    # 取出區域
    patch = image[y - half:y + half, x - half:x + half]
    
    # 計算梯度
    Gx = conv2d(patch, SOBEL_X, padding='same')
    Gy = conv2d(patch, SOBEL_Y, padding='same')
    
    magnitude = np.sqrt(Gx**2 + Gy**2)
    direction = np.arctan2(Gy, Gx)
    direction = np.rad2deg(direction) % 360  # 0 to 360
    
    # 減去主方向（旋轉不變性）
    direction = (direction - orientation) % 360
    
    # 對每個子區域計算直方圖
    descriptor = []
    bin_width = 360 / num_bins
    
    for i in range(grid_size):
        for j in range(grid_size):
            # 子區域範圍
            y0 = i * cell_size
            y1 = (i + 1) * cell_size
            x0 = j * cell_size
            x1 = (j + 1) * cell_size
            
            cell_mag = magnitude[y0:y1, x0:x1]
            cell_dir = direction[y0:y1, x0:x1]
            
            # 計算直方圖
            hist = np.zeros(num_bins)
            for cy in range(cell_size):
                for cx in range(cell_size):
                    bin_idx = int(cell_dir[cy, cx] / bin_width) % num_bins
                    hist[bin_idx] += cell_mag[cy, cx]
            
            descriptor.extend(hist)
    
    # 正規化
    descriptor = np.array(descriptor)
    norm = np.linalg.norm(descriptor)
    if norm > 1e-5:
        descriptor = descriptor / norm
    
    # Clip（SIFT 的技巧：限制最大值）
    descriptor = np.minimum(descriptor, 0.2)
    
    # 再次正規化
    norm = np.linalg.norm(descriptor)
    if norm > 1e-5:
        descriptor = descriptor / norm
    
    return descriptor

# 測試
if len(keypoints) > 0:
    octave, scale, y, x = keypoints[0]
    img = pyramid[octave][scale]
    ori, _ = compute_keypoint_orientation(img, y, x)
    desc = compute_sift_descriptor(img, y, x, ori)
    
    print(f"描述子維度：{len(desc)}")
    print(f"描述子範例（前 16 維）：{desc[:16]}")
    
    # 視覺化
    plt.figure(figsize=(12, 3))
    plt.bar(range(len(desc)), desc)
    plt.xlabel('Dimension')
    plt.ylabel('Value')
    plt.title('SIFT Descriptor (128D)')
    plt.show()

---

# Part 7: 完整的簡化 SIFT 流程

In [ ]:
def sift_simplified(image, num_octaves=4, num_scales=5, contrast_threshold=0.02):
    """
    簡化版 SIFT。
    
    Returns
    -------
    keypoints_with_descriptors : list of dict
        每個關鍵點包含：
        - octave, scale, y, x: 位置
        - orientation: 主方向
        - descriptor: 128 維描述子
    """
    # Step 1: 建構 Gaussian Pyramid
    pyramid, sigmas = build_gaussian_pyramid(image, num_octaves, num_scales)
    
    # Step 2: 計算 DoG
    dog = compute_dog(pyramid)
    
    # Step 3: 找極值點
    keypoints = find_extrema(dog, contrast_threshold)
    
    # Step 4: 計算方向和描述子
    results = []
    for octave, scale, y, x in keypoints:
        img = pyramid[octave][scale]
        
        # 計算方向
        orientation, _ = compute_keypoint_orientation(img, y, x)
        
        # 計算描述子
        descriptor = compute_sift_descriptor(img, y, x, orientation)
        
        results.append({
            'octave': octave,
            'scale': scale,
            'y': y,
            'x': x,
            'orientation': orientation,
            'descriptor': descriptor
        })
    
    return results

# 執行
sift_keypoints = sift_simplified(test_image)
print(f"偵測到 {len(sift_keypoints)} 個 SIFT 關鍵點")

# 顯示幾個關鍵點的資訊
for i, kp in enumerate(sift_keypoints[:5]):
    print(f"  {i}: O{kp['octave']} S{kp['scale']} ({kp['y']}, {kp['x']}) ori={kp['orientation']:.1f}°")

---

# 總結

## SIFT 核心概念

| 步驟 | 目的 | 方法 |
|------|------|------|
| Gaussian Pyramid | 建立尺度空間 | 不同 σ 的 Gaussian blur |
| DoG | 偵測 blob 結構 | 相鄰尺度相減 |
| Extrema Detection | 找關鍵點 | 26-鄰域極值 |
| Orientation | 旋轉不變性 | 梯度方向直方圖 |
| Descriptor | 描述外觀 | 4×4×8 = 128 維 |

## 簡化 vs 完整 SIFT

完整 SIFT 還包括：
- 精確關鍵點位置（亞像素精度）
- 邊緣響應消除
- 更複雜的尺度計算
- 更多的後處理

## 下一步

在 03_hog.ipynb 中，我們將學習 **HOG** 特徵，它更適合描述物體形狀。

In [ ]:
# 最終驗證
print("=== 函數驗證 ===")

test_funcs = [
    ("build_gaussian_pyramid(image)", lambda: build_gaussian_pyramid(test_image)),
    ("compute_dog(pyramid)", lambda: compute_dog(pyramid)),
    ("find_extrema(dog)", lambda: find_extrema(dog)),
    ("compute_keypoint_orientation(image, y, x)", 
     lambda: compute_keypoint_orientation(pyramid[0][0], 50, 50)),
    ("compute_sift_descriptor(image, y, x, ori)", 
     lambda: compute_sift_descriptor(pyramid[0][0], 50, 50, 0)),
    ("sift_simplified(image)", lambda: sift_simplified(test_image)),
]

for name, func in test_funcs:
    try:
        result = func()
        print(f"✓ {name}")
    except Exception as e:
        print(f"✗ {name}: {e}")

print("\n所有函數驗證完成！")

---

# 練習題

### 練習 1：特徵點匹配 (Feature Matching)

SIFT 最重要的應用是找兩張圖片中的對應點。

**任務**：實作 `match_descriptors` 函數，用最近鄰演算法匹配兩組描述子。

**提示**：
1. 對每個描述子 A，找最接近的描述子 B
2. 使用 Lowe's ratio test：如果最近距離 / 次近距離 < 0.75，才接受匹配
3. 距離使用歐氏距離

In [ ]:
# 練習 1 解答：特徵點匹配

def match_descriptors(desc1, desc2, ratio_threshold=0.75):
    """
    使用最近鄰 + Lowe's ratio test 匹配描述子。
    
    Parameters
    ----------
    desc1 : list of np.ndarray
        第一組描述子（N 個）
    desc2 : list of np.ndarray
        第二組描述子（M 個）
    ratio_threshold : float
        Lowe's ratio 閾值（通常 0.75-0.8）
    
    Returns
    -------
    matches : list of (i, j)
        匹配對，i 是 desc1 的索引，j 是 desc2 的索引
    """
    matches = []
    
    for i, d1 in enumerate(desc1):
        # 計算與所有 desc2 的距離
        distances = []
        for j, d2 in enumerate(desc2):
            dist = np.linalg.norm(d1 - d2)
            distances.append((dist, j))
        
        # 排序找最近的兩個
        distances.sort(key=lambda x: x[0])
        
        if len(distances) < 2:
            continue
        
        best_dist, best_idx = distances[0]
        second_dist, _ = distances[1]
        
        # Lowe's ratio test
        if second_dist > 1e-6 and best_dist / second_dist < ratio_threshold:
            matches.append((i, best_idx))
    
    return matches


# 測試：建立兩張相似的圖片
def create_transformed_image(image, shift=(10, 10), noise_level=5):
    """建立位移+雜訊版本的圖片"""
    H, W = image.shape
    shifted = np.zeros_like(image)
    sy, sx = shift
    
    # 位移
    shifted[max(0,sy):min(H,H+sy), max(0,sx):min(W,W+sx)] = \
        image[max(0,-sy):min(H,H-sy), max(0,-sx):min(W,W-sx)]
    
    # 加入雜訊
    noise = np.random.randn(H, W) * noise_level
    shifted = np.clip(shifted + noise, 0, 255)
    
    return shifted

# 建立測試圖片對
image1 = test_image.copy()
image2 = create_transformed_image(test_image, shift=(15, 20), noise_level=8)

# 提取 SIFT 特徵
kps1 = sift_simplified(image1, contrast_threshold=0.03)
kps2 = sift_simplified(image2, contrast_threshold=0.03)

print(f"Image 1: {len(kps1)} keypoints")
print(f"Image 2: {len(kps2)} keypoints")

# 提取描述子
desc1 = [kp['descriptor'] for kp in kps1]
desc2 = [kp['descriptor'] for kp in kps2]

# 匹配
matches = match_descriptors(desc1, desc2, ratio_threshold=0.75)
print(f"Matches found: {len(matches)}")

In [ ]:
# 視覺化匹配結果
def visualize_matches(img1, kps1, img2, kps2, matches, max_matches=20):
    """
    視覺化特徵點匹配結果。
    """
    # 並排兩張圖
    H1, W1 = img1.shape
    H2, W2 = img2.shape
    H = max(H1, H2)
    combined = np.zeros((H, W1 + W2))
    combined[:H1, :W1] = img1
    combined[:H2, W1:] = img2
    
    plt.figure(figsize=(14, 7))
    plt.imshow(combined, cmap='gray')
    
    # 畫連線
    colors = plt.cm.rainbow(np.linspace(0, 1, min(len(matches), max_matches)))
    
    for idx, (i, j) in enumerate(matches[:max_matches]):
        kp1 = kps1[i]
        kp2 = kps2[j]
        
        # 座標換算到原圖
        scale1 = 2 ** kp1['octave']
        scale2 = 2 ** kp2['octave']
        
        y1, x1 = kp1['y'] * scale1, kp1['x'] * scale1
        y2, x2 = kp2['y'] * scale2, kp2['x'] * scale2 + W1  # 右邊圖片偏移
        
        plt.plot([x1, x2], [y1, y2], '-', color=colors[idx], linewidth=1.5)
        plt.plot(x1, y1, 'o', color=colors[idx], markersize=5)
        plt.plot(x2, y2, 'o', color=colors[idx], markersize=5)
    
    plt.title(f'Feature Matching: {len(matches)} matches found (showing {min(len(matches), max_matches)})')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# 顯示匹配結果
if len(matches) > 0:
    visualize_matches(image1, kps1, image2, kps2, matches)
else:
    print("沒有找到匹配，可能需要調低 ratio_threshold 或 contrast_threshold")

### 練習 2：尺度不變性測試

驗證 SIFT 的尺度不變性：同一個物體縮放後，應該仍能找到對應的關鍵點。

**任務**：
1. 將測試圖片縮放到 0.5x 和 2x
2. 在不同尺度的圖片上偵測關鍵點
3. 嘗試匹配不同尺度間的關鍵點

In [ ]:
# 練習 2 解答：尺度不變性測試

def resize_image(image, scale):
    """
    簡單的雙線性插值縮放。
    """
    H, W = image.shape
    new_H, new_W = int(H * scale), int(W * scale)
    
    # 建立座標映射
    y_new = np.arange(new_H)
    x_new = np.arange(new_W)
    
    # 映射回原始座標
    y_orig = y_new / scale
    x_orig = x_new / scale
    
    # 雙線性插值
    output = np.zeros((new_H, new_W))
    
    for i, yo in enumerate(y_orig):
        for j, xo in enumerate(x_orig):
            y0 = int(np.floor(yo))
            x0 = int(np.floor(xo))
            y1 = min(y0 + 1, H - 1)
            x1 = min(x0 + 1, W - 1)
            
            fy = yo - y0
            fx = xo - x0
            
            output[i, j] = (
                (1 - fy) * (1 - fx) * image[y0, x0] +
                (1 - fy) * fx * image[y0, x1] +
                fy * (1 - fx) * image[y1, x0] +
                fy * fx * image[y1, x1]
            )
    
    return output

# 建立不同尺度的圖片
img_original = test_image.copy()
img_half = resize_image(test_image, 0.5)
img_double = resize_image(test_image, 2.0)

print(f"Original: {img_original.shape}")
print(f"Half: {img_half.shape}")
print(f"Double: {img_double.shape}")

# 在各尺度偵測關鍵點
kps_orig = sift_simplified(img_original, contrast_threshold=0.02)
kps_half = sift_simplified(img_half, contrast_threshold=0.02)
kps_double = sift_simplified(img_double, contrast_threshold=0.02)

print(f"\nKeypoints:")
print(f"  Original: {len(kps_orig)}")
print(f"  Half size: {len(kps_half)}")
print(f"  Double size: {len(kps_double)}")

In [ ]:
# 視覺化不同尺度的關鍵點分布
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

def draw_keypoints(ax, image, keypoints, title):
    ax.imshow(image, cmap='gray')
    colors = ['red', 'green', 'blue', 'yellow']
    for kp in keypoints:
        scale_factor = 2 ** kp['octave']
        y = kp['y'] * scale_factor
        x = kp['x'] * scale_factor
        r = (kp['scale'] + 1) * scale_factor
        color = colors[kp['octave'] % len(colors)]
        circle = plt.Circle((x, y), r, fill=False, color=color, linewidth=1)
        ax.add_patch(circle)
    ax.set_title(f'{title}\n{len(keypoints)} keypoints')
    ax.axis('off')

draw_keypoints(axes[0], img_half, kps_half, 'Half Size (0.5x)')
draw_keypoints(axes[1], img_original, kps_orig, 'Original (1x)')
draw_keypoints(axes[2], img_double, kps_double, 'Double Size (2x)')

plt.suptitle('Scale Invariance Test: Keypoints at Different Scales')
plt.tight_layout()
plt.show()

# 嘗試匹配 original vs double
desc_orig = [kp['descriptor'] for kp in kps_orig]
desc_double = [kp['descriptor'] for kp in kps_double]

matches_scale = match_descriptors(desc_orig, desc_double, ratio_threshold=0.8)
print(f"\nMatches between Original and Double: {len(matches_scale)}")

if len(matches_scale) > 0:
    print("\n尺度不變性驗證：即使圖片放大 2 倍，SIFT 仍能找到對應的關鍵點！")
else:
    print("\n（注意：簡化版 SIFT 在尺度差異大時可能表現不佳，完整版會更好）")

### 練習 3：消除邊緣響應 (Edge Response Elimination)

DoG 會對邊緣產生很強的響應，但邊緣點不是好的關鍵點。

SIFT 用 Hessian 矩陣的主曲率比來過濾：

$$
\frac{\text{Tr}(H)^2}{\text{Det}(H)} < \frac{(r+1)^2}{r}
$$

其中 $r$ 是主曲率比的閾值（通常 r=10）。

**任務**：實作 `eliminate_edge_responses` 函數

In [ ]:
# 練習 3 解答：消除邊緣響應

def compute_hessian_ratio(dog, octave, scale, y, x):
    """
    計算 Hessian 矩陣的 trace^2 / det 比值。
    
    Hessian 矩陣：
    H = [[Dxx, Dxy],
         [Dxy, Dyy]]
    
    trace = Dxx + Dyy
    det = Dxx * Dyy - Dxy^2
    """
    img = dog[octave][scale]
    H, W = img.shape
    
    # 邊界檢查
    if y <= 0 or y >= H-1 or x <= 0 or x >= W-1:
        return float('inf')
    
    # 二階偏導數（使用中心差分）
    Dxx = img[y, x+1] + img[y, x-1] - 2 * img[y, x]
    Dyy = img[y+1, x] + img[y-1, x] - 2 * img[y, x]
    Dxy = (img[y+1, x+1] - img[y+1, x-1] - img[y-1, x+1] + img[y-1, x-1]) / 4
    
    trace = Dxx + Dyy
    det = Dxx * Dyy - Dxy ** 2
    
    # 避免除以零
    if det <= 0:
        return float('inf')
    
    return trace ** 2 / det


def eliminate_edge_responses(keypoints, dog, r=10):
    """
    消除邊緣響應，只保留 blob-like 的關鍵點。
    
    Parameters
    ----------
    keypoints : list of (octave, scale, y, x)
    dog : DoG pyramid
    r : float
        主曲率比閾值（越小越嚴格）
    
    Returns
    -------
    filtered : list of (octave, scale, y, x)
    """
    threshold = (r + 1) ** 2 / r
    filtered = []
    
    for octave, scale, y, x in keypoints:
        ratio = compute_hessian_ratio(dog, octave, scale, y, x)
        
        if ratio < threshold:
            filtered.append((octave, scale, y, x))
    
    return filtered


# 測試邊緣消除效果
print("邊緣響應消除測試：")
print(f"  原始關鍵點數：{len(keypoints)}")

filtered_kps = eliminate_edge_responses(keypoints, dog, r=10)
print(f"  r=10 後：{len(filtered_kps)} ({len(keypoints)-len(filtered_kps)} removed)")

filtered_kps_strict = eliminate_edge_responses(keypoints, dog, r=5)
print(f"  r=5 後：{len(filtered_kps_strict)} ({len(keypoints)-len(filtered_kps_strict)} removed)")

In [ ]:
# 視覺化邊緣消除效果
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

def visualize_kps(ax, image, kps_list, title):
    ax.imshow(image, cmap='gray', vmin=0, vmax=255)
    colors = ['red', 'green', 'blue', 'yellow']
    for octave, scale, y, x in kps_list:
        scale_factor = 2 ** octave
        orig_y = y * scale_factor
        orig_x = x * scale_factor
        r = (scale + 1) * scale_factor * 2
        color = colors[octave % len(colors)]
        circle = plt.Circle((orig_x, orig_y), r, fill=False, color=color, linewidth=1)
        ax.add_patch(circle)
    ax.set_title(f'{title}\n({len(kps_list)} keypoints)')
    ax.axis('off')

visualize_kps(axes[0], test_image, keypoints, 'Original (no filtering)')
visualize_kps(axes[1], test_image, filtered_kps, 'After edge elimination (r=10)')
visualize_kps(axes[2], test_image, filtered_kps_strict, 'Strict filtering (r=5)')

plt.suptitle('Edge Response Elimination Effect')
plt.tight_layout()
plt.show()

print("\n觀察：")
print("- 消除邊緣響應後，保留的關鍵點更集中在 corner/blob 區域")
print("- r 值越小，過濾越嚴格")
print("- 完整 SIFT 還會做亞像素精度的關鍵點定位")